In [1]:
from nltk.corpus import stopwords
import pandas as pd
import re
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [2]:
train = pd.read_csv('EI-oc-En-joy-train.txt', sep='\t', error_bad_lines=False)
train.columns = ["ID", "Tweet", "Affect_Dimension", "Intensity_Class"]
dev = pd.read_csv('EI-oc-En-joy-dev.txt', sep='\t', error_bad_lines=False)
dev.columns = ["ID", "Tweet", "Affect_Dimension", "Intensity_Class"]
test= pd.read_csv("EI-oc-En-joy-test.txt", sep='\t', error_bad_lines=False)
test.columns = ["ID", "Tweet", "Affect_Dimension", "Intensity_Class"]

<ipython-input-2-0e23d5bb97fc>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train = pd.read_csv('EI-oc-En-joy-train.txt', sep='\t', error_bad_lines=False)
<ipython-input-2-0e23d5bb97fc>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dev = pd.read_csv('EI-oc-En-joy-dev.txt', sep='\t', error_bad_lines=False)
<ipython-input-2-0e23d5bb97fc>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test= pd.read_csv("EI-oc-En-joy-test.txt", sep='\t', error_bad_lines=False)


In [3]:
test

,ID,Tweet,Affect_Dimension,Intensity_Class
0,2018-En-00290,People are truly #amazing. #inspiring day,joy,NONE
1,2018-En-03093,What are some good #funny #entertaining #inter...,joy,NONE
2,2018-En-00588,@NoahWebHouse A review of my book faulted me f...,joy,NONE
3,2018-En-01532,#good to learning #wisdom &lt;&lt; reform (v):...,joy,NONE
4,2018-En-03258,"Really excited to see our team this year, and ...",joy,NONE
...,...,...,...,...
1100,2018-En-01638,@LFC_Povey I'm enjoying this so much😂 they're ...,joy,NONE
1101,2018-En-00202,"Chanyeol always cheering, he'll claps loudly a...",joy,NONE
1102,2018-En-04354,"And yet again, @MarliePacker sets the back row...",joy,NONE
1103,2018-En-02244,themselves so don't let them get to you. #hate...,joy,NONE


In [4]:
train['Intensity_Class'] = pd.Categorical(train['Intensity_Class'])
train['Intensity_Class'] =train.Intensity_Class.cat.codes

dev['Intensity_Class'] = pd.Categorical(dev['Intensity_Class'])
dev['Intensity_Class'] = dev.Intensity_Class.cat.codes

test['Intensity_Class'] = pd.Categorical(test['Intensity_Class'])
test['Intensity_Class'] = test.Intensity_Class.cat.codes

In [5]:
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stop_words.update(["now","let", 'zero', 'one', 'two', 'three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)

#Removing Stop Words

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

train['Tweet'] = train['Tweet'].apply(removeStopWords)
test['Tweet'] = test['Tweet'].apply(removeStopWords)
dev['Tweet'] = dev['Tweet'].apply(removeStopWords)

#Stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

train['Tweet'] = train['Tweet'].apply(stemming)
dev['Tweet'] = dev['Tweet'].apply(stemming)
test['Tweet'] = test['Tweet'].apply(stemming)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from keras.utils import to_categorical
target=train.Intensity_Class.values
y=to_categorical(target)

target=dev.Intensity_Class.values
c=to_categorical(target)

max_features = 13000
max_words = 50
batch_size = 128
epochs = 3
num_classes=4

X_train = train["Tweet"]
X_val = dev["Tweet"]
Y_train = y
Y_val = c

In [7]:
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,GRU,LSTM,Embedding
from keras.optimizers import Adam
from keras.layers import SpatialDropout1D,Dropout,Bidirectional,Conv1D,GlobalMaxPooling1D,MaxPooling1D,Flatten
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [9]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from keras.preprocessing.text import Tokenizer

In [14]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

In [15]:
X_test = tokenizer.texts_to_sequences(test['Tweet'])
X_test =pad_sequences(X_test, maxlen=max_words)

In [16]:
X_train =pad_sequences(X_train, maxlen=max_words)
X_val = pad_sequences(X_val, maxlen=max_words)
X_test =pad_sequences(X_test, maxlen=max_words)

In [17]:
model3_LSTM=Sequential()
model3_LSTM.add(Embedding(max_features,100,mask_zero=True))
model3_LSTM.add(LSTM(64,dropout=0.4,return_sequences=True))
model3_LSTM.add(LSTM(32,dropout=0.5,return_sequences=False))
model3_LSTM.add(Dense(num_classes,activation='sigmoid'))
model3_LSTM.compile(loss='binary_crossentropy',optimizer=Adam(lr = 0.001),metrics=['accuracy'])
model3_LSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1300000   
                                                                 
 lstm (LSTM)                 (None, None, 64)          42240     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 4)                 132       
                                                                 
Total params: 1,354,788
Trainable params: 1,354,788
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [18]:
history3=model3_LSTM.fit(X_train, Y_train, validation_data=(X_val, Y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/3
13/13 [==============================] - 16s 452ms/step - loss: 0.6802 - accuracy: 0.2754 - val_loss: 0.6644 - val_accuracy: 0.1931
Epoch 2/3
13/13 [==============================] - 3s 216ms/step - loss: 0.6099 - accuracy: 0.2778 - val_loss: 0.6042 - val_accuracy: 0.1966
Epoch 3/3
13/13 [==============================] - 4s 293ms/step - loss: 0.5720 - accuracy: 0.3391 - val_loss: 0.6037 - val_accuracy: 0.1897


In [19]:
#y_pred3=model3_LSTM.predict_classes(X_test, verbose=1)
import numpy as np
predict_x=model3_LSTM.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

classes_x

35/35 [==============================] - 4s 30ms/step


array([0, 0, 0, ..., 0, 0, 0])